<h1><center><b>Logistic Regression Model on Flights Dataset</b></center></h2>

<h3><center>Name: Irvin Carreon</center></h3>

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
import pandas as pd
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import scipy as sp
import seaborn as sns
import numpy as np

In [ ]:
flights = pd.read_csv("D:\\Irvin\\top_ports_lines.csv")

In [ ]:
flights['cancelled'] = flights['cancelled'].map({1:0, 2:1})

In [ ]:
flights["carrier_delay"] = flights["carrier_delay"].fillna(0)
flights["weather_delay"] = flights["weather_delay"].fillna(0)
flights["nas_delay"] = flights["nas_delay"].fillna(0)
flights["security_delay"] = flights["security_delay"].fillna(0)
flights["late_aircraft_delay"] = flights["late_aircraft_delay"].fillna(0)
flights["dep_delay"] = flights['dep_delay'].fillna(flights["dep_delay"].median())

In [ ]:
flights['carrier_delay'] = flights['carrier_delay'].astype(int)
flights['weather_delay'] = flights['weather_delay'].astype(int)
flights['nas_delay'] = flights['nas_delay'].astype(int)
flights['security_delay'] = flights['security_delay'].astype(int)
flights['late_aircraft_delay'] = flights['late_aircraft_delay'].astype(int)
flights['dep_delay'] = flights['dep_delay'].astype(int)

In [ ]:
flights['cancelled'].value_counts()/flights.shape[0]

In [ ]:
le = LabelEncoder()

flights["operating_airline_encoded"] = le.fit_transform(flights['operating_airline'])

encoding_map = flights[["operating_airline", "operating_airline_encoded"]]

In [ ]:
le2 = LabelEncoder()

flights['origin_encoded'] = le.fit_transform(flights['origin'])

encoding_map2 = flights[['origin', 'origin_encoded']]

In [ ]:
flights['crs_dep_time'] = flights['crs_dep_time'].astype(str)

In [ ]:
hours = []

for row in flights['crs_dep_time']:
    if len(row)==3:
        hours.append(row[:1])
    elif len(row)==4:
        hours.append(row[:2])
    else:
        hours.append("0")

flights["dep_hour"] = hours

In [ ]:
flights.to_csv("flights_for_R.csv", index=False)

In [ ]:
flights2 = flights[['operating_airline_encoded', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'origin_encoded', 'dep_hour', 'dep_delay', 'distance']]

In [ ]:
x = flights[['operating_airline_encoded', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'origin_encoded', 'dep_hour', 'dep_delay', 'distance']]
y = flights[['cancelled']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .5, random_state = 123)
x_validation, x_test, y_validation, y_test = train_test_split(x, y, test_size = .2, random_state = 123)
print(x_train.shape)
print(y_train.shape)
print(x_validation.shape)
print(y_validation.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
log_model = LogisticRegression(random_state = 123, class_weight='balanced')
log_model.fit(x_train,y_train)

In [ ]:
print(log_model.coef_)

In [ ]:
y_pred_val = log_model.predict(x_validation)

In [ ]:
accuracy_score(y_validation,y_pred_val)

In [ ]:
f1_score(y_validation, y_pred_val, average='binary')

<h3><center><b>Synthetic Minority Over Sampling Technique</b></center></h3>

In [ ]:
#Apply SMOTE
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=123)

x_res, y_res = sm.fit_resample(x_train, y_train)

In [ ]:
log_model_res = LogisticRegression(random_state = 123)
log_model_res.fit(x_res, y_res)

In [ ]:
y_pred_val_res = log_model_res.predict(x_validation)

In [ ]:
accuracy_score(y_validation,y_pred_val_res)

In [ ]:
f1_score(y_validation, y_pred_val_res, average='binary')

In [ ]:
sp.stats.pearsonr(flights['cancelled'], flights['dep_delay'])

flights_plot = flights[['cancelled', 'dep_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']]
sns.pairplot(flights_plot)

In [ ]:
flights_plot = flights[['cancelled', 'dep_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']]
np.corrcoef(flights_plot, rowvar=False)